# MuMoT User Manual <a class="tocSkip">

## Multiscale Modelling Tool  <a class="tocSkip">

This is the user manual for [MuMoT](https://github.com/DiODeProject/MuMoT), a software tool developed at the University of Sheffield as part of the [DiODe](http://diode.group.shef.ac.uk) project

*Current MuMoT version*: **0.0**

### Working with MuMoT

MuMoT runs inside [Jupyter notebooks](http://jupyter.org) - since you are reading this User Manual you have presumably already installed Jupyter, or are using an installation provided to you

Next, you need to install MuMoT itself; since MuMoT is a Python package it can be installed from PyPi - see the [project page](https://diodeproject.github.io/MuMoT/) for more details

To run commands in 'Code' cells it's usually easiest just to hit `Shift + Enter`; here is some basic information on [working with Jupyter notebooks](http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html)

With Jupyter and MuMoT both installed, you simply need to import the MuMoT package into your notebook; we recommend you refer to it as `mmt`

In [ ]:
import MuMoT.MuMoT as mmt

### Defining your first model

If everything went well with importing MuMoT you should have seen the message

``Created `%%model` as an alias for `%%latex``

MuMoT models are defined using the `%%model` keyword within a cell, and have a very simple syntax; since MuMoT works with *chemical kinetic* or *reaction kinetic* models, you simply have to describe the reactions that take place, and the rates with which they occur. For example

`A + A -> A + U: s`

is a very simple reaction, in which two particles of type $A$ interact, and one changes to type $U$, at rate $s$.

These kind of reaction rules are quite general; they describe a lot of chemical reactions, as well as collective behaviour in which individuals interact to change each other's state, and even demographic models in which individuals are born and die.

Our first model will be based on signalling behaviour observed in honeybee swarms, as described in Seeley et al. ([2012](#references)) and Pais et al. ([2013](#references)).


The `$` at the start and end just allow us to use LaTeX [codes](http://reu.dimacs.rutgers.edu/Symbols.pdf) for our state and rate labels, allowing us to use  letters from different alphabets, such as Greek, and other nice formatting - if you don't know LaTeX don't worry about this - leave in the `$` signs, they won't hurt.

**N.B.** currently it is important to put spaces between all elements of each rule, as below.

OK, here we go...

In [ ]:
%%model
$
U -> A : g_A
U -> B : g_B
A -> U : a_A
B -> U : a_B
A + U -> A + A : r_A
B + U -> B + B : r_B
A + B -> A + U : s
A + B -> B + U : s
$

Now we have our model, and Jupyter should have reprinted the rules underneath the cell you input them in.

You can see that actually the `_` code enables you to use subscripts, which can be handy for distinguishing related rates - in this model there are two distinct nest sites for the honeybee swarm to choose between, *A* and *B*, and rates associated with each one.

We have our model, but currently it is not useful; it's only a definition, not a fully-fledged model that we can start analysing with MuMoT. To make one of those we have to use the `parseModel` command; we simply pass in a reference to the cell where we defined our model - this should be in input cell 2...

In [ ]:
model1 = mmt.parseModel(In[2])

If there is no output from `parseModel` that is good news - it means the model was valid. Now we have an object, which we called `model1`, which is a version of the model that we can start doing useful work with. For example, we can see its 'reactants' (or states, or types of individual; remember the terminology comes from chemical reactions)...

In [ ]:
model1.showReactants()

...and the rates the reactions happen at

In [ ]:
model1.showRates()

Note that here MuMoT also helpfully tells us which reactions in the original model definition each rate is associated with.

We can also look at the reaction equations in a slightly nicer format...

In [ ]:
model1.show()

We can even see a simple figure representing our model...

In [ ]:
model1.visualise()

This figure attempts to represent the nature of the interactions between reactants graphically, from simple transitions (arrows), inhibitions (filled circles) and induced switches (arrows with filled circles at the origin); to understand the different interaction patterns try relating the arrows with the reactions we defined above, by reference to the rates they are labelled with.

### Exploring your new model

If you want to know what commands you can send to an object, Jupyter lets you type out the object name, then a `.`, then hit `Tab` to see a list of accepted commands. You can also consult the [Command reference](#comreference) for a full list of commands, and details on their use.

One commmand that looks intruiguing is `showODEs`

In [ ]:
model1.showODEs()

This is where things start getting interesting, because equations are what mathematicians analyse. `showODEs` shows the model's Ordinary Differential Equations, which describe how the populations of reactants in the model change over time. MuMoT has derived these equations for you automatically, from your description of the reactions. If you are familiar with ODEs then you should be able to read these quite easily. However even if you are not, you can still work with them.

So, what analysis can we do in MuMoT now we've found our ODEs?

Well, we might want to look at how the state of the system (the proportions of the different reactants) change on average over time...

...but before we can do that, we need to let MuMoT know that this is actually a simpler set of equations than it appears to be. $A$, $B$, and $U$ represent states that honebees can be in during the swarming process, either committed to site *A*, committed to site *B*, or *U*ncommitted. Because there is a constant number of individuals in the swarm during a decision, one of these equations is redundant; the change in the uncommitted bee frequency can be worked out from the change in frequencies of A-committed and B-committed bees, for example.

We let MuMoT know this by defining one of the reactants in terms of the other, using `substitute`

In [ ]:
model2 = model1.substitute('U = N - A - B')

Now we have a simplified model, `model2`, which only has two equations, because we have told MuMoT that the total number of uncommitted bees $U$ is the total number of bees in the swarm $N$, minus the numbers committed to the two different nests ($A$ and $B$)

In [ ]:
model2.showODEs()

Now we can look at how the model behaves, by asking for a vector plot of the two reactants $A$ and $B$...

In [ ]:
vector1 = model2.vector('A', 'B')

Now we can see how to start working with models in MuMoT. The figure you see is a vector plot, which shows on average how the state of the system (proportions of bees in states $A$ and $B$) will change over time, from any point in the state space. The arrows give the direction the system will move in, and their lengths show how fast. Because the total number of bees is constant, $A+B$ can never exceed 1, so the top right hand triangle of the figure is greyed out; these are impossible states. The number of uncommitted bees $U$, which we hid away earlier, is not shown, but it is simply $1-A-B$. But what we really care about in a decision-making process is how the number of 'voters' for the competing options changes over time.

You will also notice that there are some sliders above the figure; MuMoT gives you a slider for every rate in the model, and by changing their values, you can see what happens to the behaviour of the system.

Jupyter also lets you interact nicely with the figure; you can zoom in, back out, pan around, and save, all by clicking the icons that appear underneath it. If you want to save a figure for a paper, simply click the disk icon. And if while exploring the model you find a nice figure, you can click the 'off' icon in its top right to snapshot it, and generate a new interactive figure to play with. You can then simply right-click your snapshots to save them from your browser - just don't forget to note the parameters that generated them - you can find that out by looking at the logs... later in the manual we will see how to take notes properly to reproduce analyses (see [Bookmarking](#bookmarking)).

Every 'controller' in MuMoT keeps a log of everything it's been asked to do; we called our controller `vector1` when we created it, so it is easy to check what it has been up to... (WARNING: if you have been playing with the sliders a lot, there could be a lot in the logs! To only see the last few entries we set the `tail` argument to `True`; if `tail` is not set it will be assumed to be `False`).

In [ ]:
vector1.showLogs(tail = True)

For papers and presentations we can produce somewhat prettier plots similar to figure 1 above, using `stream`. But before we do that, one unfortunate thing about our `vector1` controller is that there are just too many sliders. We might choose to describe all the rates (*a*bandonment, *r*ecruitment (via the waggle dance), and individual discovery and commitment (*g*)) in terms of the *v*alues of the nest sites... we can do this by using `substitute` again:

In [ ]:
model3 = model2.substitute('a_A = 1/v_A, a_B = 1/v_B, g_A = v_A, g_B = v_B, r_A = v_A, r_B = v_B')

Our new model is a little simpler, with just three rates, $v_A$, $v_B$ and $s$.

In [ ]:
model3.show()

Still, in decision-making what we really care about is how good options are on average, and their difference in value. Pais et al. (2013) thus defined option values in terms of deviation from the average value, so
$$\displaystyle v_A=\mu+\frac{\Delta}{2}$$
and
$$\displaystyle v_B=\mu-\frac{\Delta}{2}$$

In [ ]:
model4 = model3.substitute('v_A = \mu + \Delta/2, v_B = \mu - \Delta/2')
model4.showODEs()

This model is much easier to interact with...

In [ ]:
stream1 = model4.stream('A','B', showFixedPoints = True)

A `stream` plot is just like a `vector` plot, except now lines show the average change of the system over time in more detail, and their shading represents the speed of change, from slow (light grey) to fast (black).

Note also that here we added a *keyword argument* `showFixedPoints = True` - this can also be applied to `vector` plots, and plots the stationary points of the equation. If the stationary point is *stable* (arrows converge on that point) then it is represented with a filled circle, and if it is *unstable* (arrows leave that point) then a hollow circle is shown. Displaying fixed points can slow down plots, so by default they are not shown, and you need to explicitly request they are included.

Analysis of fixed points is an important technique for understanding *dynamical systems* such as our model. But while exploring a model interactively by changing its parameters and seeing how it behaves can help develop intuition, really we need a more systematic approach. Fortunately we can do this via *bifurcation plots*, using the `bifurcation` command...

`bifurcation` takes two arguments, the first is the parameter we want to vary systematically, the second is the state we want to observe.

In [ ]:
bifurcation1 = model4.bifurcation('s','A-B')

Our new bifurcation plot lets us see what happens to fixed points in the system as we vary a control parameter, in this case the cross-inhibitory stop-signalling rate between bees, $s$. We have also chosen not to observe a single reactant as our state variable, but rather the difference between two of them; this makes sense because in decision-making we care about the difference in votes for options

Our plot shows the location of fixed points, and they are either depicted as *stable* (solid line) or *unstable* (dashed line). Where the stability of fixed points changes, or fixed points appear of dissappear, these are referred to as *bifurcation points*. Using the interactive `bifurcation` plot above see if you can reproduce the bifurcation diagrams shown in figure 5(i) and 5(ii) of [Pais et al. (2013)](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0073216)

Figure 5(iii) of [Pais et al. (2013)](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0073216) requires us to specify a different bifurcation parameter...

In [ ]:
bifurcation2 = model4.bifurcation('\\Delta','A-B')

### The effects of finite system size

The ODE models we have been looking at based on our model are very useful, but they represent an idealised scenario; they describe the behaviour of a system with infinitely many components or, to put it another way, the expected behaviour of the system, minus any noise.

Noise is very important in collective behavour systems, however, and can have a variety of causes. The first cause we will consider is noisy fluctuations from the infinite population ideal, caused by having a finite, much smaller, population. To begin analysing the noise in our system, dependent on its size, we can deploy two main techniques; stochastic simulation, and statistical physics analysis.

We will return to stochastic simulation later, concentrating first on statistical physics analysis. As you might imagine, the maths involved in a statistic physics analysis is very advanced, based on deriving and then analysing the *Master Equation*; fortunately MuMoT automates this analysis for you.

Brace yourself, this might look intimidating...

In [ ]:
model1.showMasterEquation()

You really don't need to understand the equation above, but it's nice to know that MuMoT generated it for you, isn't it? The Master Equation describes the probabilities of different transitions between system states. We are particularly interested in it because we can use it to derive the infinite population behaviour of the system (the ODEs we already saw), but also the noise around that. This is done by applying a technique called the *van Kampen expansion* [van Kampen, 2007)](#references), to approximate the *Focker-Planck equation*. In the *Advanced users* section at the end of this manual you'll be able to look at these equations, if you dare, to convince yourself that MuMoT is doing some clever stuff for you

But what is this clever stuff useful for? Well, for one thing, it lets us see approximately what the effects of noise, caused by finite system size, are in terms of deviations from the fixed points of the system...

In [ ]:
stream2 = model4.stream('A','B', showFixedPoints = True, showNoise = True)

This new `stream` plot now lets us see how the system's expected state gets 'spread out' around the fixed points. Slide the system size slider to see how the extent of the spread depends on the number of components in the system; slide the other sliders to see how the shape of the noise changes, in a way we might not have guessed, as the parameters change.

The effects of noise in the `stream` plot above are based on <s>mathematical approximations</s> the computational approximations of the Master Equation. When the system has less one or two reactant the noise effects can be mathematically approximated from the Master Equation. The computational approximation is achieved through the efficient and accurate *Stochastic Simulation Algorithm* (SSA) (or *Doob-Gillespie algorithm*) [(Gillespie, 1976)](#references). Alternatively, MuMot gives access to the SSA simulator to simply visualise the behaviour of the system over time.

To run the SSA simulations on your model, you can use the command `SSA()`. This command allow you to run a single simulation run, you can specify several advanced parameters from the _"Advanced options"_ tab, as described in the [Command reference](#comreference) section.
Try modifying the parameters and see how they change your results.

In [ ]:
ssa = model4.SSA(initWidgets={'initialState': {'U': [1,0,1,0.05], 'B': [0,0,1,0.05], 'A': [0,0,1,0.05]},
                              'systemSize':[50,5,100,1] })

### Spatial noise

MuMoT also enables more sophisticated stochastic simulations of your model. The `multiagent` command allows you to run simulations either with static agents interacting over a static communication network, or mobile agents communicating to neighbours within a local range of communication.

Each agent represents a reactant, and exchanges messages to interact with other agents and change its internal state  (described as a *probabilistic finite state machine*). The conversion of transition rates into the probabilities is done following the methodology proposed in Reinal et al. ([2005](#references)).

The multiagent simulations have several _Advanced options_, as described in the [Command reference](#comreference) section. Try modifying the parameters to see how they change your results.

In [ ]:
ssa = model4.multiagent(initWidgets={'initialState': {'U': [1,0,1,0.05], 'B': [0,0,1,0.05], 'A': [0,0,1,0.05]},
                              'systemSize':[20,5,100,1], 'visualisationType':'graph' })

### Going to a higher dimension
Our first model ended up being 2-dimensional, after we realised that there were a constant number of components in the system. However MuMoT can work with higher-dimensional models (although not all features are supported for them; see [Supported features](#features)).

### Non-constant system size
Not all models are of the kind studied above, in which there are a constant number of components. In open systems components can be created and destroyed, so the total size of the system can fluctuate over time, and in principle has no limits.

A classic model from both ecology and chemistry is the *Lotka-Volterra* model; ecologists often think of this as representing two interacting populations, one of predators, and one of prey. Individuals are born and die in both populations. MuMoT enables this both via the `\emptyset` notation $\emptyset$, in which components spontaneously arise from nothing, or are destroyed, and via steady-state reactant populations, denoted with `()`, which do not change over time but simply persist at a constant size.

Our Lotka-Volterra example is taken from Murray ([2002](#references), p124). Individuals are born into the prey population $X$ at a rate that depends on that population size, then predatory interactions decrease the prey population size and increase the predator population size. Finally, predators experience background mortality.

In [ ]:
%%model
$
(A) + X -> X + X : \alpha
X + Y -> Y + Y : \beta
Y -> \emptyset : \gamma
$

In [ ]:
model5 = mmt.parseModel(In[23])
model5.show()

In [ ]:
model5.showODEs()

Note that there is no equation for $A$; since it is specified to be at a constant concentration, there is no equation for $A$'s change over time. This means that when we try and interact with the model, we will need to specify the constant value that $A$ should take.

Also, since the system size is now no longer constant, as discussed above, it is now useful to be able to change the limits on the plot, to ensure that all the interesting behaviour of the system is visible...

In [ ]:
stream2 = model5.stream('X','Y', showFixedPoints = False)

### Bookmarking
<a id='bookmarking'></a>

If you find an interesting parameter set and want to preserve it, you can store those parameter values in a form that lets you reproduce your analysis. Simply click on the bookmark button underneath the sliders; you should see a message that a bookmark has been pasted to the logs, and instructions how to view it. Let's follow these instructions from our latest plot... click the bookmark button, then run the following command:

In [ ]:
stream2.showLogs(tail = True)

Now we need simply copy the last line of the log file, paste into a new cell, and replace `<modelName>` with the model we are working with, in this case `model5`; this is easily done by clicking in the cell then using *Find and Replace* from the *Edit* menu. We will also give our new bookmarked view a unique name, so we can refer to it if we need to in the future...

In [ ]:
bookmark1 = model5.stream('X', 'Y', params = [('\\beta', 0.8), ('\\alpha', 0.5), ('A', 0.5), ('\\gamma', 0.5), ('plotLimits', 3.5), ('systemSize', 1)], showFixedPoints = False, bookmark = False)

### Command reference
<a id='comreference'></a>

#### Supported features
<a id='features'></a>
##### Graphical output
<a id='graphout'></a>
<table>
  <tr>
    <th>Command</th>
    <th>1d-system</th> 
    <th>2d-system</th> 
    <th>3d-system</th>
    <th>empty-set</th>
    <th>constant reactants</th>
  </tr>
  <tr>
    <td><code>stream</code></td>
    <td>&#10005;</td> 
    <td>&#10003;</td> 
    <td>&#10005;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>vector</code></td>
    <td>&#10005;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>SSA</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>multiagent</code><br>(static network)<br>(moving particles)</td>
    <td><br>&#10003;<br>&#10003;</td> 
    <td><br>&#10003;<br>&#10003;</td> 
    <td><br>&#10003;<br>&#10003;</td>
    <td><br>&#10005;<br>&#10003;</td>
    <td><br>&#10005;<br>&#10003;</td>
  </tr>
  <tr>
      <td><code>stream</code><br>(analytical noise)<br>(computational noise)</td>
    <td><br>&#10003;<br>&#10005;</td> 
    <td><br>&#10005;<br>&#10003;</td> 
    <td><br>&#10005;<br>&#10003;</td>
    <td><br>&#10005;<br>&#10003;</td>
    <td><br>&#10005;<br>&#10003;</td>
  </tr>
  <tr>
    <td><code>bifurcation</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10005;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>integrate</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
   <tr>
    <td><code>noiseCorrelations</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
</table>

##### Display of mathematical formulas
<a id='mathout'></a>
<table>
  <tr>
    <th>Command</th>
    <th>1d-system</th> 
    <th>2d-system</th> 
    <th>3d-system</th>
    <th>4d-system</th> 
    <th>empty-set</th>
    <th>constant reactants</th>
  </tr>
  <tr>
    <td><code>showODEs</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>showODEs_vKE</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>showMasterEquation</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>showVanKampenExpansion</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>showFokkerPlanckEquation</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>showNoiseEOM</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>showNoiseStationarySol</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
</table>

##### Model properties
<a id='modelprop'></a>
<table>
  <tr>
    <th>Command</th>
    <th>1d-system</th> 
    <th>2d-system</th> 
    <th>3d-system</th>
    <th>4d-system</th> 
    <th>empty-set</th>
    <th>constant reactants</th>
  </tr>
  <tr>
    <td><code>show</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>showRates</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>showReactants</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>showConstantReactants</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>showStoichiometry</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>visualise</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
</table>

#### Getting help
You can get help on commands by using the `help()` command. To get basic information on MuMoT, for example, type

In [ ]:
help(mmt)

To get help on a specific command, such as the `stream` command, available for a model, type

In [ ]:
help(model1.stream)

Optional arguments are given with the default value they will have if not supplied

Optional keywords may also be given; again the default value they will have if not supplied is given

All controller-generating commands have the optional argument `params`, which contains a list of fixed parameter values (see [Partial controllers](#partialcont))

In addition, all plotting functions accept a number of formatting keywords, as follows:

* TODO

#### Advanced options for `SSA`

* proportion of each reactant at timestep zero. Remember that the sum of all (non-constant) reactants must be 1.0
* simulation time is the length of each simulation run
* random seed which is a number to control (and repeat) random stochastic effects. Any run that uses the same random seed will have tha same identical random fluctuations
* flag to plot proportions (i.e. reactants are in closed interval [0,1]) or full populations (i.e. reactants are in closed interval [0,systemSize])
* flag to plot results in realtime. If True, the plot is updated each timestep of the simulation; if False, the plot is updated once at the end of the simulation
* number of simulation runs to be executed
* flag to aggregate or not the results from several runs
* type of visualisation::
  * Temporal evolution with y-axis showing the population size of each reactant varying as a function of the time on the x-axis. When aggregated, the results are visualised as boxplots. 
  * Final distribution of the population among the reactants. In this visualisation, the use can specify the reactants on x and y axes.  When aggregated, the results are visualised as an ellipse estimated as the covariance matrix
  * Barplot showing the final distribution. When aggregated, the barplots show the average and the error bars show the standard deviation; otherwise the results of the last simulation run are displayed

(Note that modifying the widgets: type of visualisation, flag to plot proportions, and flag to aggregate does not trigger the computation of new data. All other widgets do.)

#### Advanced options for `multiagent`
`multiagent` views have all the options described above for the `SSA` simulations and, in addition:
* type of network. Available types are:
  * Full graph; a complete network in which every agent communicates with every agent
  * Random network with Erdos-Renyi topology. The widgets allow to specify:
    * the linking probability with which each agent has a communication edge with another agent
  * Random network with Barabasi-Albert topology. The widgets allow to specify:
    * the number of edges every new node (during creation) is linked to
  * Simulation of moving particles in which agent move in a periodic space and interact locally. The widgets allow to specify:
    * range of interaction of the agent
    * motion correlatedness of the agent. Correlatedness = 1 correponds to straight movement, Correlatedness = 0 corresponds to uncorrelated random walk.
    * speed of the agent, i.e. displacement in one timestep
    * flag to show the motion trajectory of the agents
    * flag to show interactions; agents within communication range are liked through a line
* length of one timestep; the maximum timestep is determined by the rates

(Note that in addition to the one specified for the `SSA` simulations, here the widgets to show motion trajectory and to show interactions do not trigger the recomputation of the simulations).
Static networks are available only for models that do not contain the empty-set or constant reactant, whereas moving particles have no constraints; see [*Supported features*](#features).

#### Advanced options for `bifurcation`
* Initial &#60;&#60;name of bifurcation parameter&#62;&#62;:
  * Here you can choose the starting value of the bifurcation parameter. MuMoT tries to determine all stationary states and starts numerical continuation (using PyDSTool) from all stable fixed points found.
* Reactant &#60;&#60;name of reactant&#62;&#62; at t=0:
  * This will only have an effect if no stationary solutions could be detected. Then these initial conditions will be passed on to PyDSTool for the numerical continuation.
* Note: Currently only 1D and 2D ODE systems are supported in `bifurcation` and only Branch Points (BP) and Limit Points (LP) can be detected in 1-parameter bifurcations (or more adequately termed codimension one bifurcations).

#### Advanced options for `integrate` and `noiseCorrelations`
* Reactant &#60;&#60;name of reactant&#62;&#62; at t=0:
  * Determine the initial conditions for all reactants in the systems. Note that if `conserved = True` occurs as a keyword argument in `integrate` and `noiseCorrelations` then initial conditions cannot be set independently. 
* Simulation time: 
  * Determines the length of the simulation.
* Plot proportions (only in `integrate`): 
  * Choose if absolute numbers or proportions are displayed (check box). Proportions are denoted $\Phi$ with the reactant as index. If true, the absolute number is divided by the total number of reactants (system size) at t=0. If the system is conserved, the sum of all concentrations sum up to 1 for all t.

### Advanced users

#### Partial controllers
<a id='partialcont'></a>
If you are creating a notebook to demonstrate a concept, it may be easiest to fix some of the parameters to ensure that the user can only vary the parameters you want them to. Partial controllers can be created by passing the standard commands a list `params` of parameter name, parameter value pairs. For example...

In [ ]:
partial1 = model4.stream('A','B', params = [('\\mu',1),('s',1)], showFixedPoints = False)

#### Viewing the Master and Fokker-Planck equations, and calculating & displaying noise

Just to make sure you really want to look at these, and because the size of the equations can slow down your browser, some of the following commands are commented. Uncomment then run if you want to see the gory details...

To show this functionality we make use of a model describing the production and decay of protein $P$ and its dimerization into $P_2$ (more conveniently called $X$ and $Y$ below), see Hayot & Jayaprakash ([2004](#references)) for details. This model is defined as:

In [ ]:
%%model
$
\emptyset -> X : k_3
X -> \emptyset : k
X + X -> Y + \emptyset : k_1
Y + \emptyset -> X + X : k_2
$

As you already know it from the examples above, let's parse the model ...

In [ ]:
model6 = mmt.parseModel(In[32])

Now that MuMoT knows the model, we can continue deriving the <b>chemical Master equation</b>. 

Master equations describe the time evolution of a system that, in general, can be in one of many different possible states at a given point in time with a certain probability. Over time, the system makes transitions so that the overall state changes continuously. Transitions are governed by transition probabilities that are usually summarised in a transition matrix. The Master equation is a set of first-order differential equations that describe the evolution of the state dependent probabilities driven by the transition matrix.

The <b>chemical Master equation</b> ([van Kampen, 2007](#references)) is a master equation for the probabilities that the system has any given composition of rectants as a function of time. In MuMoT this equation can be derived using:

In [ ]:
model6.showMasterEquation()

Here, P(X,Y,t) describes the probability to observe the systems's composition of reactants X and Y at time t. In MuMoT, the system size that, at this point, is not specified and which is used as formal parameter for the system size expansion described below is denoted $\overline{V}$. Please pay attention to this notation when defining your model using the letter $V$ (but don't worry, MuMoT will not confuse both variables).

Here we used step operators $E_{op}$ to express the chemical Master equation in a compact form. Note that we have implemented the derivation of the Master equation as it was originally done in [van Kampen (2007)](#references).

Unfortunately, there is only a limited number of simple cases that allow to solve the Master equation exactly. Hence, analytical approximations may help, such as the system size expansion (also called van Kampen expansion), which is automatised in MuMoT using `showVanKampenExpansion`. 

The idea of this method is to expand the Master equation in powers of a small parameter, which is
<br>
$$\sim \frac{1}{\sqrt{system\, size}} \,.$$
<br>

Let's do the expansion with MuMoT and display it for the current model ($\overline{V}$ = system size):

In [ ]:
#model6.showVanKampenExpansion()

This equation describes an expansion in the parameter $\overline{V}$, which represents the system size. We repeat that at this point this is not specified. However, often $\overline{V}$ is a volume or the total number of particles in a real system. For more information on the system size expansion we refer to [van Kampen (2007)](#references).

Collecting all terms $\propto \sqrt{\overline{V}}$ gives the ODEs, which can be displayed using:

In [ ]:
model6.showODEs_vKE()

Here, we have introduced $\Phi_{X,Y}$, which describe the concentrations of $X$ and $Y$ in the mixture (i.e. $\Phi_{X}=X/\overline{V}$, for example). We note that the time-dependent quantities in the ODEs correspond to concentrations when derived via van Kampen expansion.

Next, if we collect all terms $\propto \overline{V}$ we obtain the Fokker-Planck equation in the linear noise approximation:

In [ ]:
model6.showFokkerPlanckEquation()

This equation describes the probability of fluctuations $\eta_{X,Y}$ at time t. In MuMoT, fluctuations are automatically denoted $\eta$ with the index denoting the reactant. These quantities describe the fluctuations around the mean field value of the concentrations $\Phi_{X,Y}$. 

Having this equation allows us derive the equations of motion for the fluctuations. This can be done using:

In [ ]:
model6.showNoiseEOM()

Here, we have first-order moments, $\langle \eta_{X}\rangle$ and $\langle \eta_{Y}\rangle$, and second-order moments, $\langle \eta_{R_1}\eta_{R_2}\rangle$, where $R_{1,2}=X,Y$.

We can analytically derive the stationary solutions of these equations and display them using:

In [ ]:
model6.showNoiseStationarySol()

Another possibility to study noise in the system is by calculating the time-dependent noise-noise correlations according to:

In [ ]:
nc1 = model6.noiseCorrelations(initWidgets={'maxTime':[20,5,50,1]})

These correlation functions relate to fluctuations around a fixed point. To see the corresponding fixed point we can simply show the logs:

In [ ]:
nc1.showLogs()

Please keep this stationary state of the system in mind for a little moment.

In MuMoT, we can also numerically integrate our dynamical system (here model6). To do this, we use the `integrate` method as follows: 

In [ ]:
int1 = model6.integrate(initWidgets={'maxTime':[30,5,50,1]})

Now click on the *Advanced options* tab above the figure and tick the *Plot population proportions* tab. We then display the last point on the curve by showing the logs with the following command:

In [ ]:
int1.showLogs()

Here, you can see the last point on the curve before selecting to plot proportions and after choosing this option. Remember, the proportions are calculated by dividing the total numbers by the system size at t=0 (which, if you haven't changed anything, should equal 10 in the plot above -- see the *System size* slider above the figure). 

Please note that the values on the sliders are the same as in the plot of the noise-noise correlations above. If you could memorise the fixed point around which the correlation functions were plotted (if not, it was ($\Phi_X=1$, $\Phi_Y=1$)) and compare this with the last point on the curve above (see `int1.showLogs` above), we can safely say that both are in agreement. So, it works! The noise-noise correlations have been plotted around the stationary state of the dynamical system. Using `integrate` we obtained a graphical confirmation of this.

Finally, we would like to mention that `integrate` can be used as soon as the model has been defined and parsed. It is also useful independent of studying noise in the system.

### References
<a id='references'></a>
* Gillespie, D. T. (1976). A general method for numerically simulating the stochastic time evolution of coupled chemical reactions. *Journal of Computational Physics* **22** (4): 403–434
* Hayot, F. & C. Jayaprakash (2004) 
[The linear noise approximation for molecular fluctuations within cells](http://iopscience.iop.org/article/10.1088/1478-3967/1/4/002/pdf). *Physical Biology* **1**, 205-210
* Murray, J. D. (2002) *Mathematical Biology I. An Introduction (Second Edition)*. Springer
* Pais, D., Hogan, P.M., Schlegel, T., Franks, N.R., Leonard, N.E. & Marshall, J.A.R. (2013) [A mechanism for value-sensitive decision-making](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0073216).  *PLoS one* **8**(9), e73216
* Reina A., Valentini G., Fernández-Oto A., Dorigo M. & Trianni V. (2015) [A design pattern for decentralised decision making](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0140950).  *PLoS one* **10**(10), e0140950
* Seeley, T.D, Visscher, P.K. Schlegel, T., Hogan, P.M., Franks, N.R. & Marshall, J.A.R. (2012) [Stop signals provide cross inhibition in collective decision-making by honeybee swarms](http://www.sciencemag.org/content/335/6064/108.full.pdf). *Science* **335**, 108-111
* van Kampen, N. (2007) *Stochastic Processes in Physics and Chemistry (Third Edition)*. North-Holland